This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 8; Grid Count = 8; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 15:12:58Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:20:52 PM	705aac94...
#1: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:16:31 PM	fe883d23...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:14:05 PM	cab2c050...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#5: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:13:56 PM	643f6a58...
#6: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...
#7: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 3:12:58 PM	13d0abde...


In [5]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

705aac94-c204-413b-bf1b-8d68dd3f7310

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [8]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("iFlx0") &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#2: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 3:12:58 PM	13d0abde...


# A Helper Function to obtain different plot formats

In [9]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [10]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M500.1,57.1 L553.5,57.1 M45.6,113.1 L51.1,114.7 L56.6,115.8 L62.2,117.6 L67.7,124.5 L73.2,133.8
 L78.7,137.0 L84.3,138.3 L89.8,140.8 L95.3,145.7 L100.8,153.0 L106.3,152.9 L111.9,153.1 L117.4,153.1
 L122.9,153.1 L128.4,153.1 L134.0,153.0 L139.5,153.0 L145.0,153.0 L150.5,153.0 L156.1,153.0 L161.6,153.0
 L167.1,153.0 L172.6,153.0 L178.1,153.0 L183.7,153.0 L189.2,153.0 L194.7,153.0 L200.2,153.0 L205.8,153.0
 L211.3,153.0 L216.8,153.0 L222.3,153.0 L227.8,153.0 L233.4,153.0 L238.9,153.0 L244.4,153.0 L249.9,153.0
 L255.5,153.0 L261.0,153.0 L266.5,153.0 L272.0,153.0 L277.5,153.0 L283.1,153.0 L288.6,153.0 L294.1,153.0
 L299.6,153.0 L305.2,153.0 L310.7,153.0 L316.2,153.0 L321.7,153.0 L327.2,153.0 L332.8,153.0 L338.3,153.0
 L343.8,153.0 L349.3,153.0 L354.9,153.0 L360.4,153.0 L365.9,166.6 L371.4,177.6 L377.0,190.8 L382.5,191.0
 L388.0,191.0 L393.5,203.5 L399.0,219.4 L404.6,239.2 L410.1,285.5 L415.6,304.6 L421.1,307.6 L426.7,307.7
 L432.2,307.7 L437.7,307.8 L443.2,307.7 L448.7,307.7 L454.3,307.8 L459.8,307.8 L465.3,307.8 L470.8,307.7
 L476.4,307.8 L481.9,307.8 L487.4,307.8 L492.9,307.8 L498.4,307.8 L504.0,307.8 L509.5,307.8 L515.0,307.8
 L520.5,307.8 L526.1,307.8 L531.6,307.8 L537.1,307.8 L542.6,307.8 L548.1,307.8 L553.7,307.8 L559.2,307.8
 L564.7,307.8 L570.2,307.8 L575.8,307.8 L581.3,307.8 L586.8,307.8 L592.3,307.8 L597.9,307.8 L603.4,307.8
 L608.9,307.8 L614.4,307.8 L619.9,307.8 L625.5,307.8 L631.0,307.8 L636.5,307.8 L642.0,307.8 L647.6,307.8
 L653.1,307.8 L658.6,307.8 L664.1,307.8 L669.6,307.8 L675.2,307.8 L680.7,307.8 L686.2,307.8 L691.7,307.8
 L697.3,307.8 L702.8,307.8 L708.3,307.8 L713.8,307.8 L719.3,307.8 L724.9,307.8 L730.4,307.8 L735.9,307.8
 L741.4,307.8 L747.0,378.1 L752.5,378.6 L758.0,384.9 L763.5,384.9 L769.0,385.0 L774.6,385.0 L780.1,385.1
 L785.6,385.1 L791.1,385.1 L796.7,385.1 L802.2,385.1 L807.7,385.1 L813.2,385.1 L818.8,385.2 L824.3,385.1
 L829.8,385.1 L835.3,385.1 L840.8,385.1 L846.4,385.1 L851.9,385.1 L857.4,385.1 L862.9,385.1 L868.5,385.1
 L874.0,385.1 L879.5,385.1 L885.0,385.1 L890.5,385.1 L896.1,385.1 L901.6,385.1 L907.1,385.1 L912.6,385.1
 L918.2,385.1 L923.7,385.1 L929.2,385.1 L934.7,385.1 L940.2,385.1 L945.8,385.1 L951.3,385.1 L956.8,385.1
 L962.3,385.1 L967.9,385.1 L973.4,385.1 L978.9,385.1 L984.4,385.1 L989.9,385.1 L995.5,385.1 L1001.0,385.1
 L1006.5,385.1 L1012.0,385.1 L1017.6,385.1 L1023.1,385.1 L1028.6,385.1 L1034.1,385.1 L1039.7,440.2 L1045.2,439.7
 L1050.7,439.7 L1056.2,439.8 L1061.7,439.8 L1067.3,439.8 L1072.8,439.8 L1078.3,439.8 L1083.8,439.8 L1089.4,439.8
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M500.1,81.1 L553.5,81.1 M4

## Residual History

In [11]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M11.3,57.1 L64.7,57.1 M45.6,83.2 L49.1,85.8 L52.6,86.8 L56.2,88.3 L59.7,96.7 L63.2,107.4
 L66.7,116.8 L70.3,118.3 L73.8,109.1 L77.3,116.9 L80.8,128.9 L84.3,173.4 L87.9,195.0 L91.4,185.2
 L94.9,213.3 L98.4,220.5 L102.0,216.3 L105.5,222.9 L109.0,220.5 L112.5,225.6 L116.1,224.8 L119.6,228.8
 L123.1,220.0 L126.6,227.1 L130.1,232.5 L133.7,233.5 L137.2,236.0 L140.7,239.3 L144.2,240.3 L147.8,245.4
 L151.3,247.5 L154.8,247.3 L158.3,252.8 L161.8,254.3 L165.4,255.0 L168.9,256.9 L172.4,257.1 L175.9,258.0
 L179.5,258.4 L183.0,258.9 L186.5,260.0 L190.0,260.1 L193.5,260.4 L197.1,260.4 L200.6,263.3 L204.1,263.3
 L207.6,263.3 L211.2,263.6 L214.7,265.3 L218.2,265.7 L221.7,265.8 L225.2,265.9 L228.8,265.9 L232.3,265.9
 L235.8,265.9 L239.3,265.9 L242.9,265.9 L246.4,265.9 L249.9,91.3 L253.4,101.4 L257.0,108.0 L260.5,109.4
 L264.0,109.5 L267.5,110.7 L271.0,115.6 L274.6,137.5 L278.1,170.3 L281.6,197.7 L285.1,232.1 L288.7,244.3
 L292.2,255.5 L295.7,262.2 L299.2,274.3 L302.7,277.4 L306.3,281.6 L309.8,281.6 L313.3,284.5 L316.8,285.6
 L320.4,288.5 L323.9,291.5 L327.4,292.8 L330.9,294.1 L334.4,294.9 L338.0,296.4 L341.5,297.2 L345.0,297.4
 L348.5,297.5 L352.1,297.7 L355.6,298.9 L359.1,299.9 L362.6,300.4 L366.1,300.3 L369.7,300.7 L373.2,300.9
 L376.7,301.9 L380.2,301.9 L383.8,302.4 L387.3,302.7 L390.8,303.2 L394.3,303.2 L397.9,303.2 L401.4,303.3
 L404.9,303.5 L408.4,303.6 L411.9,303.9 L415.5,304.1 L419.0,304.2 L422.5,304.6 L426.0,304.3 L429.6,304.6
 L433.1,304.8 L436.6,305.2 L440.1,305.2 L443.6,305.4 L447.2,305.4 L450.7,305.5 L454.2,305.5 L457.7,305.5
 L461.3,305.6 L464.8,305.6 L468.3,305.6 L471.8,305.7 L475.3,305.7 L478.9,305.7 L482.4,305.8 L485.9,305.8
 L489.4,305.8 L493.0,161.7 L496.5,231.7 L500.0,264.1 L503.5,285.1 L507.0,292.4 L510.6,300.4 L514.1,316.8
 L517.6,319.4 L521.1,323.0 L524.7,331.0 L528.2,334.6 L531.7,333.1 L535.2,338.6 L538.8,339.2 L542.3,339.5
 L545.8,343.3 L549.3,343.2 L552.8,344.9 L556.4,348.5 L559.9,348.7 L563.4,348.9 L566.9,349.8 L570.5,350.4
 L574.0,351.3 L577.5,351.5 L581.0,351.6 L584.5,352.0 L588.1,352.0 L591.6,352.2 L595.1,352.4 L598.6,353.2
 L602.2,353.3 L605.7,353.3 L609.2,353.4 L612.7,353.6 L616.2,353.6 L619.8,353.5 L623.3,353.9 L626.8,354.0
 L630.3,354.1 L633.9,354.3 L637.4,354.3 L640.9,354.0 L644.4,354.0 L647.9,354.4 L651.5,354.6 L655.0,354.6
 L658.5,354.6 L662.0,354.7 L665.6,354.7 L669.1,354.7 L672.6,354.7 L676.1,354.6 L679.7,283.4 L683.2,304.4
 L686.7,319.4 L690.2,320.3 L693.7,331.7 L697.3,336.0 L700.8,334.1 L704.3,339.6 L707.8,337.0 L711.4,339.8
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M11.3,81.1 L64.7,81.1 M45.6,54.5 L49.1,56.0 L52

save results to text file

In [12]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [13]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 3:12:58 PM	13d0abde...


Number of cells

In [14]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280, 5120 ]

helper arrays

In [24]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(4,ses.Count());


Function used to calculate the EOCs

In [25]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

In [26]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

In [27]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(4,ses.Count()+1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(5,ses.Count()+1);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.WriteLine($"P={p},sess={iSess}");
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -8 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [28]:
errorsAndCells

Dimension,2
Lengths,"[ 4, 5 ]"
Storage,"[ 0.44721359549995787, 0.023940855986105646, 0.0032144940105413734, 0.0008750016444413799, 0.00037782501865665476, 0.22360679774997894, 0.0134594325396264, 0.0006497499506191623, 0.0001259844706090987, 3.481094613539112E-05, 0.11180339887498954, 0.0070032045051262055, 0.00020757010904109995, 3.572685002353764E-05, 1.0309590300542182E-05, 0.05590169943749477, 0.003653535388862935, 5.869069500295124E-05, 6.389593967553816E-06, 1.3209668603809664E-06 ]"
IsContinuous,True
StructureType,General
Length,20
NoOfCols,5
NoOfRows,4
IsLocked,False


In [29]:
EOCs

[ 0.9040365496187611, 1.9251053533593216, 2.3658058871068572, 2.719993384285038 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [21]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlux0"));
sessions

In [22]:
var si = sessions.Pick(0);
si

Error: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'index')
   at System.Linq.ThrowHelper.ThrowArgumentOutOfRangeException(ExceptionArgument argument)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at Submission#23.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

plot it

In [23]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=5;
var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
foreach(var timestep in si.Timesteps){
    if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
            IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
            var SF = lts.GetShadowFields();
            var texplot = new Tecplot(SF[0].GridDat, 2);
            texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
    }
}

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#24.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)